## Import Library

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE 
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, recall_score, precision_score, accuracy_score

## Load dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dhykac/Telco-Customer-Churn/main/Telco%20Customer%20Churn.csv')
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


# Data Preprocessing

## Blank Values Handling and Encoding Phase

In [4]:
# Drop customer ID which is unique per customers
df.drop('customerID', axis=1, inplace=True)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


If we check df.info() , the `TotalCharges` column had dtypes object because the column had blank column. So we need to replace blank space with `0`.

In [5]:
# Get rid of anomalies by replace it by 0
df['TotalCharges'] = df['TotalCharges'].apply(lambda x:0 if x ==' ' else x)
df['TotalCharges'] = df['TotalCharges'].astype(float)

Now, let's get a new dataframe with dummies from categorical feature.

In [6]:
df = pd.get_dummies(df, drop_first=True, dtype=int)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,1,29.85,29.85,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,0,34,56.95,1889.50,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0,2,53.85,108.15,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
3,0,45,42.30,1840.75,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,0,2,70.70,151.65,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   gender_Male                            7043 non-null   int64  
 5   Partner_Yes                            7043 non-null   int64  
 6   Dependents_Yes                         7043 non-null   int64  
 7   PhoneService_Yes                       7043 non-null   int64  
 8   MultipleLines_No phone service         7043 non-null   int64  
 9   MultipleLines_Yes                      7043 non-null   int64  
 10  InternetService_Fiber optic            7043 non-null   int64  
 11  Inte

## Imbalanced Handling

In [8]:
# View the percentage of target feature
target = df['Churn_Yes'].value_counts().reset_index()
target.rename(columns={'index':'Churn','Churn_Yes':'freq'}, inplace=True)
target['percentage'] = round((target['freq']/target['freq'].sum())*100,2)
target

,Churn,freq,percentage
0,0,5174,73.46
1,1,1869,26.54


Churn had frequency 73:26 , my personal opinion the data considered imbalancing for clean & tidy data like telco customer churn (which is different from real & lively moving industry data). So we need to handle imbalanced data and make it balanced. And also, because if we go for undersampling the data will lose too much information (about 3305 data). So we go for oversampling with SMOTE.

In [9]:
# Use oversampling with SMOTE to data
from imblearn import under_sampling, over_sampling
sm = over_sampling.SMOTE(random_state=22)

X = df.drop(['Churn_Yes'],axis = 1)
Y = df['Churn_Yes']
X_sm, y_sm = sm.fit_resample(X, Y)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100

Shape of X before SMOTE: (7043, 30)
Shape of X after SMOTE: (10348, 30)

Balance of positive and negative classes (%):


1    50.0
0    50.0
Name: Churn_Yes, dtype: float64

Now, data had been balancing. We could go to the next steps.

# Modeling 

## Split Data set with percentage 80:20

In [10]:
# Data percentage train 80 : test 20
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=22)

## Random Forest modeling

In [11]:
# Machine Learning using random forest
model = RandomForestClassifier(random_state=22)
model.fit(X_train, y_train)
preds = model.predict(X_test)

## Evaluation

In [12]:
# Calculate Accuracy (ONLY VALID FOR BALANCED DATA)
print('Accuracy',accuracy_score(y_test, preds))

Accuracy 0.8420289855072464


In term of Accuracy, the model got an accuracy 84.20% chance to success.

In [13]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, preds)
cm

array([[890, 174],
       [153, 853]])

The confusion matrix show :

* The model predicted churned and the customer actually churned is 890.
* The model predicted not churned and the customer actually churned is 174.
* The model predicted churned and the customer actually not churned is 153.
* The model predicted not churned and the customer actually not churned is 853.

In [14]:
# calculate AUC
print('AUC ',roc_auc_score(y_test, preds))

# calculate F1 Score
print('F1 Score ',f1_score(y_test, preds))

# calculate Precision
print('Precision ',precision_score(y_test, preds))

# calculate Recall
print('Recall ',recall_score(y_test, preds))

AUC  0.8421893451322142
F1 Score  0.8391539596655189
Precision  0.8305744888023369
Recall  0.8479125248508946


* AUC : The model succeed to distinct between True Positive and True Negative with chance 84.22%
* F1 Score : the harmonic mean between precission and recall is 83.92% which is important for us to consider False Positive and False Negative.
* Precission : The rate of model predict results are False Positive (which is 17%)
* Recall : The rate of model predict results are False Negative (which is 15%)

In this kind of data, we must concern on False Negative. This is because we should put our focus on the customer which is predicted to stay with company but actually Churned. So the better Recall score, the better company will be prepared.

# Manual Check

In [15]:
# Manual checking phase
data_manual_check = {'y_test' : y_test, 'preds' : preds}
manual_check = pd.DataFrame(data_manual_check)

In [16]:
manual_check.head(10)

,y_test,preds
5457,0,0
8533,1,1
9145,1,1
6895,0,0
2827,0,1
8321,1,1
7365,1,1
9120,1,1
6626,1,1
3364,1,1


In [17]:
manual_check.tail(10)

,y_test,preds
4991,1,0
4384,1,1
250,0,0
5941,0,0
5536,1,0
6615,0,0
7616,1,1
10211,1,1
2149,0,0
2834,0,0


In [18]:
manual_check.sample(10)

,y_test,preds
2023,1,1
9084,1,1
2389,0,0
9644,1,1
5936,0,0
6079,0,0
5363,0,0
6764,1,1
8519,1,1
7934,1,1
